In [1]:
import argparse
import cv2
import numpy as np
from PIL import Image 
import math

## assumes input of homographized frame

1) solvepnp

notes: actual camera calibration matrix stored on pi

In [78]:
def get_pose(frame,K,tag_size=1):
    
    ## camera
    opoints = np.asarray([[-1, -1, 0],
                        [ 1, -1, 0],
                        [ 1,  1, 0],
                        [-1,  1, 0]])
    

    s = 0.5*tag_size    
    
    ## scale camera
    opoints = opoints*s
    
    
    ## replace distortion coeffecients with values from calibration
    dcoeffs = np.zeros(4)
    
    _, r, t = cv2.solvePnP(opoints, frame, K, dcoeffs,flags=cv2.SOLVEPNP_ITERATIVE)
    
    print("rotation",r,"translation",t)
    return r,t

In [7]:
K = np.asarray([[0.05, 0, 0.867745304],
                      [0, 0.05, 0.296830287], 
                      [0, 0, 1]])
dcoeffs = np.zeros(4)

2) Logic from https://github.com/YuehChuan/tb3_aprilTag/blob/c050a7451a7fd776a5d9ab9992760c40caee22f6/software/apriltags-cpp/src/CameraUtil.cpp

In [79]:
def get_pose1(fx,fy, s, frame):
    
    ## focal matrix
    F = np.identity(3)
    F[1][1] = -1
    F[2][2] = -1
    frame = np.matmul(F,frame)
    
    ## scale frame

    M = np.zeroes(4,4)
    M[0][0] =  frame[0][0] / fx
    M[0][1] =  frame[0][1]/ fx
    M[0][3] =  frame[0][2] / fx
    M[1][0] =  frame[1][0] / fy
    M[1][1] =  frame[1][1] / fy
    M[1][3] =  frame[1][2] / fy
    M[2][0] =  frame[2][0]
    M[2][1] =  frame[2][1]
    M[2][3] =  frame[2][2]

    
    ## unit vectors of columns 
    scale0 = np.sqrt(((M[0][0])**2) + ((M[1][0])**2) + ((M[2][0])**2))
    scale1 = np.sqrt(((M[0][1])**2) + ((M[1][1])**2) + ((M[2][1])**2))
    scale = np.sqrt(scale0*scale1)
    
    
    ## scale M
    M *= 1.0/scale
    
    
    ## remove anything behind camera
    if M[2][3] > 0:
        M *= -1
    
    
    ## constants
    M[3][0] = 0
    M[3][1] = 0
    M[3][2] = 0
    M[3][3] = 1

    
    ## first two columns, third rotation column is cross product  
    a= M[:,0]
    b= M[:,1]
    ab = np.cross(a,b)
    
    M[0][2] = ab[0]
    M[1][2] = ab[1]
    M[2][2] = ab[2]

    
    ## decompose current rotation and define rotation without singular values
    R = M[:3,:3]
    
    u, sigma,v = np.linalg.svd(R)
    MR = np.matmul(u,v)
    
    M[:3,:3] = MR
    
    ## scale
    for i in range(0,3):
        M[i][3] *= scl*tagSize/2
    
    return M[:3,:3],M[:,3:]